In [1]:
import pandas as pd
import re
import string
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [2]:
df = pd.read_csv('tokopediamay7.csv')

In [3]:
df.head(10)

,label,tweet
0,anger,"Soal jln Jatibaru,polisi tdk bs GERTAK gubernu..."
1,anger,"Sesama cewe lho (kayaknya), harusnya bisa lebi..."
2,happy,Kepingin gudeg mbarek Bu hj. Amad Foto dari go...
3,anger,"Jln Jatibaru,bagian dari wilayah Tn Abang.Peng..."
4,happy,"Sharing pengalaman aja, kemarin jam 18.00 bata..."
5,anger,"Dari sekian banyak thread yang aku baca, threa..."
6,happy,Sharing sama temen tuh emg guna bgt. Disaat lu...
7,sadness,Orang lain kalau pake ponco itu buat jas hujan...
8,anger,"Contoh mereka yg gemar menyudutkan, teriak pal..."
9,sadness,Pulang udah H-4 lebaran dilema sekali. Seperti...


In [4]:
'''
membuat fungsi Case Folding/Lower Case
'''
def pre_case_folding(word):
    # Untuk membuat semua tweet menjadi huruf kecil
    lower_case = word.lower()
    
    return lower_case

In [5]:
'''
membuat fungsi retweet removal
'''
def pre_retweet_removal(word):
    # Untuk menghapus karakter 'rt'
    rt_remove = re.sub(r'\brt\b\s+',"", word)

    return rt_remove

In [6]:
'''
membuat fungsi Mention Removal
'''

def pre_mention_removal(word):
    # Untuk menghilangkan mention/tag '@' pada tweet
    ment_removal = re.sub(r"@\S+", "", word)
    
    return ment_removal

In [7]:
'''
membuat fungsi hashtag Removal
'''

def pre_hashtag_removal(word):
    # Untuk menghilangkan tanda pagar beserta kata yang diikuti pada tweet
    hash_removal = re.sub(r"#\S+", "", word)
    
    return hash_removal

In [8]:
'''
membuat fungsi URLs Removal
'''
import re

def pre_urls_removal(word):
    # Untuk menghilangkan tag http di tweet
    urls_removal = re.sub(r"http\S+", "", word)
    
    return urls_removal

In [9]:
'''
membuat fungsi Non-Alpha Numeric Removal
'''

def pre_non_numeric_removal(word):
    # Untuk menghapus tanda baca yang ada pada text dengan cek setiap karakter
    non_numeric_removal = [char for char in word if char not in string.punctuation]
    # Menggabungkan setiap karakter kembali
    non_numeric_removal = ''.join(non_numeric_removal)
    # menghilangkan karakter noise lain
    non_numeric_removal = re.sub(r'\w*â€¦\w*', '', non_numeric_removal)
    non_numeric_removal = re.sub(r'\w*Â\w*', '', non_numeric_removal)
    non_numeric_removal = re.sub(r"&\S+", "", non_numeric_removal)
    
    return non_numeric_removal

In [10]:
'''
membuat fungsi Stemming
'''

# membuat fungsi stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# proses stemming
def pre_stemming(word):
    output = stemmer.stem(word)
    return output

In [11]:
'''
membuat fungsi Stopword Removal
'''

# Stoplist ditambahkan agar lebih lebih lengkap
stopwords_list = pd.read_csv("stopwordbahasa.csv", engine='python', header=None)
stopwords = r'\b(?:{})\b'.format('|'.join(stopwords_list[0].values.tolist()))


def pre_stopword(word):
    stop = re.sub(stopwords, '', word)
    no_space = stop.replace ('  ', ' ')
    return no_space

In [13]:
text = df['tweet_text']

prepro_funs = [
    pre_case_folding,
    pre_retweet_removal,
    pre_urls_removal,
    pre_mention_removal,
    pre_hashtag_removal,
    pre_non_numeric_removal,
    pre_stemming
]

KeyError: 'tweet_text'

In [15]:
for fungsi in prepro_funs:
    text = text.apply(fungsi)

In [16]:
df.loc[:,'clean'] = text

In [17]:
df.head()

,tweet_time,tweet_id,username,tweet_text,source,favourite,retweet,clean
0,2020-05-07 15:52:25,1258318759716024321,rizarobbi,https://t.co/i3jtNzhTHw,Tokopedia Twitter,0,0,
1,2020-05-07 15:52:16,1258318721161936903,WhatsApps1D,@peachbobble Beli di tokopedia,Twitter for Android,0,0,beli di tokopedia
2,2020-05-07 15:51:52,1258318619991134209,Haunz,Ide Diabetes kita hari ini: Chatime Indonesia...,Twitter for Android,0,0,ide diabetes kita hari ini chatime indonesia j...
3,2020-05-07 15:51:48,1258318607009734656,madewira,Buat temen-temenku di Denpasar yg pernah tingg...,Twitter for Android,0,0,buat tementemenku di denpasar yg pernah tingga...
4,2020-05-07 15:50:42,1258318329686552576,juurosei,@hazelzoely Aku belinya di tokopedia kak,Twitter for Android,0,0,aku beli di tokopedia kak


In [18]:
df.to_csv('tokopediamay7clean.csv', index=False)